In [1]:
! wget https://storage.googleapis.com/kaggle-forum-message-attachments/237294/7771/german_credit_data.csv

--2019-12-28 18:59:50--  https://storage.googleapis.com/kaggle-forum-message-attachments/237294/7771/german_credit_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.8.208
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.8.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53393 (52K) [text/csv]
Saving to: ‘german_credit_data.csv.1’

german_credit_data. 100%[===================>]  52.14K  --.-KB/s    in 0.01s   

2019-12-28 18:59:50 (5.14 MB/s) - ‘german_credit_data.csv.1’ saved [53393/53393]



In [2]:
import pandas as pd
df = pd.read_csv("german_credit_data.csv", index_col=0)

In [3]:
df = df.fillna(value="NA")

In [4]:
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NA,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NA,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NA,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NA,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [5]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
preprocess = make_column_transformer(
    (['Age', 'Credit amount', 'Duration'], StandardScaler()),
    (['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk'], OneHotEncoder(sparse=False))
)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:778: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


In [6]:
mat = preprocess.fit_transform(df)

In [65]:
mat[3,:]

array([0.8315025 , 1.63424694, 1.75038404, 0.        , 1.        ,
       0.        , 0.        , 1.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       1.        ])

In [15]:
X = mat[:, :-2]
Y = mat[:, -1]
print(X[10,:])

[-0.92754658 -0.70047155 -0.73866754  1.          0.          0.
  0.          1.          0.          0.          0.          1.
  0.          1.          0.          0.          0.          0.
  0.          1.          0.          0.          1.          0.
  0.          0.          0.          0.          0.        ]


In [16]:
num_feats = X.shape[1]
numX = X.shape[0]
print(num_feats)
print(numX)

29
1000


In [10]:
import numpy as np

In [11]:
datasize = 500
cs_size = 25   
split_on_doc = 0.8
testsize = 100
ratio_relevant = 0.4 # I think 0.6 will be relevant and 0.4 non-relevant
ratios_col = Y * ratio_relevant + (1-Y)*(1-ratio_relevant)

In [12]:
# generate a candidate set of size 10 everytime
data_X = []
data_Y = []
test_X = []
test_Y = []
group_identities_train = []
group_identities_test = []
print("Sampling between 0 and {} for train".format(numX*split_on_doc))
p = ratios_col[0:int(numX*split_on_doc)]
p = p / sum(p)
for i in range(datasize):
    cs_indices = np.random.choice(np.arange(0, numX*split_on_doc, dtype=int), size=cs_size, p=p)
    cs_X = X[cs_indices]
    cs_Y = Y[cs_indices]
    data_X.append(cs_X)
    data_Y.append(cs_Y)
    group_identities_train.append(cs_X[:,4])
print("Sampling between {} and {} for test".format(numX*split_on_doc, numX))
p = ratios_col[int(numX*split_on_doc):]
p = p/sum(p)
for i in range(testsize):
    cs_indices = np.random.choice(np.arange(numX*split_on_doc, numX, dtype=int), size=cs_size, p=p)
    test_X.append(X[cs_indices])
    test_Y.append(Y[cs_indices])
    group_identities_test.append(X[cs_indices,4])

Sampling between 0 and 800.0 for train
Sampling between 800.0 and 1000 for test


In [62]:
import pickle as pkl
print(len(data_X))
print(data_Y[100])
print(len(test_X[0][0]))
print(len(test_Y[0]))
pkl.dump((data_X, data_Y), open("german_train_rank_3.pkl", "wb"))
pkl.dump((test_X, test_Y), open("german_test_rank_3.pkl", "wb"))


500
[0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0.
 0.]
29
25
